In [ ]:
# hana_connection.py
from hdbcli import dbapi
import ssl
import os
from dotenv import load_dotenv  
load_dotenv()  # Load environment variables from .env file

class HanaConfig:
    HOST = "your-hana-host"        # e.g. "hana.mycorp.internal"
    PORT = 30015                   # e.g. 3NN15 on-prem / tenant port [web:130][web:132]
    USER = os.getenv("HANA_USER")
    PASSWORD = os.getenv("HANA_PASSWORD")
    PASSWORD = os
    # For HANA Cloud use encrypt=True, sslValidateCertificate=True [web:131]
    ENCRYPT = True
    SSL_VALIDATE_CERT = False      # adjust to your environment
    SSL_KEYSTORE = None            # path to trust store if needed

